## Where to Read More
This notebook will help me keep track of my experimental weighted crypto portfolio that I started for this year (2022). 

You can read more about the parameters and qualities of the portfolio in the [original blog post](https://andresberejnoi.com/best-cryptos-to-invest-in-2022/) I wrote. I write an update every month, which you can follow on my blog.

You can check my Youtube video about using the [Coinbase Pro API here](https://www.youtube.com/watch?v=a74pQbHgdXw). It's just 9 minutes long and covers a good amount of the basic actions.

### Crypto Market Data
At the moment, I'm making use of Coinbase Pro API through the [cbpro library](https://github.com/danpaquin/coinbasepro-python) to collect market data of the coins I'm tracking. The endpoint I'll be using is `https://api.exchange.coinbase.com/products/{product_id}/candles`, and the documentation can be found [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

You can also use other APIs to gather the data. I was initially using AlphaVantage with an API key, but Coinbase Pro supported more of the coins I'm tracking. 
The endpoint I would use for AlphaVantage is `DIGITAL_CURRENCY_DAILY `, and the documentation can be found [here](https://www.alphavantage.co/documentation/#currency-daily). Additionally, [this csv](https://www.alphavantage.co/digital_currency_list/) file from AlphaVantage lists all the cryptocurrencies supported by the API.


### AlphaVantage Helper Function

In [1]:
def get_daily_data(symbol, api_key, start_date=None, end_date=None, quote_currency='USD', simplify=True):
    '''Uses AlphaVantage endpoint DIGITAL_CURRENCY_DAILY to return a dataframe.
    I created this function to simplify the process'''
    
    _base_url    = "https://www.alphavantage.co/"
    api_function = 'DIGITAL_CURRENCY_DAILY'

    url = f'{_base_url}query?function={api_function}&symbol={symbol}&market={quote_currency}&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    
    #check that we received proper data
    if 'Error Message' in data:
        print(f"\t--> Error retrieving {symbol}. It's probably not supported.")
        print("\t    Returning `None`")
        return None
    
    #we need to parse data.
    _candle_data = data["Time Series (Digital Currency Daily)"]
    df = pd.DataFrame(_candle_data).transpose()  #without transpose, we get each daily candle as a column. We want each candle as a row, and each column as the OHLC data
    
    #--convert index to datetime and sort it from low to high dates
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    #AlphaVantage returns data both in USD and whatever quote currency we asked for. 
    # in case we do not want repetition, simplify will remove the fixed USD market and return a clearner dataframe just with the requested quote currency
    if simplify:
        desired_columns = ['1a. open (USD)','2a. high (USD)','3a. low (USD)','4a. close (USD)','5. volume','6. market cap (USD)']
        new_names       = ['open','high','low','close','volume','market_cap']
        
        dict_replacement = {old_name:new_name for old_name, new_name in zip(desired_columns, new_names)}
        df = df[desired_columns]
        
        df = df.rename(columns=dict_replacement)
        
    #-- in case start and end dates are specified:
    if start_date:
        if end_date:
            return df[start_date:end_date]
        else:
            return df[start_date:]
    elif end_date:
        return df[:end_date]
    
    return df

## Imports And Setup

In [2]:
import os
import pandas as pd
import requests
import time
import cbpro

from api_keys import alpha_vantage_key   #if you use AlphaVantage, put your own key here or in a different file

In [3]:
_data_folder = "./data/"
list_coins = ['BTC', 'ETH', 'BNB', 'SOL', 'ADA', 'WLUNA', 'AVAX', 'DOT', 'DOGE',
              'SHIB', 'MATIC', 'CRO', 'LTC', 'UNI', 'LINK', 'ALGO', 'BCH', 'XLM',
              'ATOM', 'AXS']

## Using Coinbase Pro Library to Get Historic Data
For the `get_historic_rates` function, the granularity for daily candles is 86400. Check the documentation [here](https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles).

In [4]:
start_date = '2021-12-23'
end_date   = '2022-06-30'

client = cbpro.PublicClient()

In [5]:
coinbase_data_individual_dfs = {}
quote_currency = 'USD'

for coin in list_coins:
    pair = f"{coin}-{quote_currency}"
    
    #cbpro will return a list of lists. We need to turn it into a dataframe
    columns = ['date','open','high','low','close','volume']
    nested_list_data = client.get_product_historic_rates(pair, start=start_date, end=end_date, granularity=86400)
    
    coinbase_data_individual_dfs[coin] = pd.DataFrame(nested_list_data, columns=columns).set_index('date',drop=True).sort_index(ascending=True)
    
    #convert date from unix time to iso format (easier to look at)
    coinbase_data_individual_dfs[coin].index = pd.to_datetime(
        coinbase_data_individual_dfs[coin].index,
        origin = 'unix',
        unit='s')

/home/andresberejnoi/miniconda3/envs/crypto_2022/lib/python3.9/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [6]:
coinbase_data_individual_dfs['BTC']

,open,high,low,close,volume
date,,,,,
2021-12-23,48032.16,51397.82,48608.61,50842.20,16627.046280
2021-12-24,50445.55,51878.60,50842.06,50851.38,11690.454182
2021-12-25,50191.84,51171.68,50852.31,50428.31,5550.571532
2021-12-26,49460.00,51295.33,50428.31,50801.79,6863.394738
2021-12-27,50480.00,52100.00,50804.33,50717.77,11596.914682
...,...,...,...,...,...
2022-06-26,20944.68,21866.00,21474.14,21027.92,11145.888527
2022-06-27,20500.00,21520.00,21024.85,20725.17,17440.942683
2022-06-28,20164.70,21189.99,20722.95,20252.89,17658.660417


### Save Full Candle Files For Each Coin
Before saving, the index is sorted so that each candle is in ascending order (ascending towards the bottom).

* ***WARNING***:
Coinbase Pro returns empty data for Binance coin (BNB). That means that we need to fix the file or dataframe before it is saved. I will use AlphaVantage API to do this since I already wrote some code for it in this notebook.

In [7]:
daily_close_coins = {}   #store only the closing data. This will be turned into a dataframe later
for i, coin in enumerate(coinbase_data_individual_dfs):
    pair = f"{coin}_{quote_currency}"
    _output_file = f"{pair}_daily_candles.csv"
    output_file = os.path.join(_data_folder, _output_file)
    
    print(f"--> Saving candles for pair: {pair}\n\tat {output_file}\n")
    
    _df = coinbase_data_individual_dfs[coin]
    
    # collect closing data
    daily_close_coins[coin] = _df['close']
    
    # write dataframe to file
    _df.to_csv(output_file, 
               index=True,
               mode='w',  #python file mode. 'a' is for append, 'w' for write, etc
               #header=False, #use this when appending, so column names are not written in the middle of the file
               )
    

--> Saving candles for pair: BTC_USD
	at ./data/BTC_USD_daily_candles.csv

--> Saving candles for pair: ETH_USD
	at ./data/ETH_USD_daily_candles.csv

--> Saving candles for pair: BNB_USD
	at ./data/BNB_USD_daily_candles.csv

--> Saving candles for pair: SOL_USD
	at ./data/SOL_USD_daily_candles.csv

--> Saving candles for pair: ADA_USD
	at ./data/ADA_USD_daily_candles.csv

--> Saving candles for pair: WLUNA_USD
	at ./data/WLUNA_USD_daily_candles.csv

--> Saving candles for pair: AVAX_USD
	at ./data/AVAX_USD_daily_candles.csv

--> Saving candles for pair: DOT_USD
	at ./data/DOT_USD_daily_candles.csv

--> Saving candles for pair: DOGE_USD
	at ./data/DOGE_USD_daily_candles.csv

--> Saving candles for pair: SHIB_USD
	at ./data/SHIB_USD_daily_candles.csv

--> Saving candles for pair: MATIC_USD
	at ./data/MATIC_USD_daily_candles.csv

--> Saving candles for pair: CRO_USD
	at ./data/CRO_USD_daily_candles.csv

--> Saving candles for pair: LTC_USD
	at ./data/LTC_USD_daily_candles.csv

--> Saving 

### Fix Missing BNB Data With AlphaVantage

In [8]:
bnb_df = get_daily_data('BNB',alpha_vantage_key, start_date=start_date, end_date=end_date, quote_currency='USD', simplify=True)
bnb_df

/home/andresberejnoi/miniconda3/envs/crypto_2022/lib/python3.9/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


,open,high,low,close,volume,market_cap
2021-12-23,533.60000000,552.60000000,524.30000000,549.00000000,844421.95400000,844421.95400000
2021-12-24,549.00000000,554.40000000,537.40000000,542.10000000,665688.41500000,665688.41500000
2021-12-25,542.10000000,550.50000000,539.10000000,547.40000000,455399.79300000,455399.79300000
2021-12-26,547.40000000,549.30000000,535.60000000,546.40000000,536338.77500000,536338.77500000
2021-12-27,546.50000000,572.00000000,543.10000000,563.70000000,803238.13100000,803238.13100000
...,...,...,...,...,...,...
2022-06-26,239.70000000,245.20000000,233.20000000,233.90000000,456762.54700000,456762.54700000
2022-06-27,234.00000000,241.00000000,230.00000000,233.80000000,483616.79800000,483616.79800000
2022-06-28,233.90000000,242.80000000,225.90000000,226.90000000,632626.77900000,632626.77900000
2022-06-29,227.00000000,229.80000000,215.40000000,219.70000000,711780.69100000,711780.69100000


In [9]:
bnb_df[start_date:end_date]

,open,high,low,close,volume,market_cap
2021-12-23,533.60000000,552.60000000,524.30000000,549.00000000,844421.95400000,844421.95400000
2021-12-24,549.00000000,554.40000000,537.40000000,542.10000000,665688.41500000,665688.41500000
2021-12-25,542.10000000,550.50000000,539.10000000,547.40000000,455399.79300000,455399.79300000
2021-12-26,547.40000000,549.30000000,535.60000000,546.40000000,536338.77500000,536338.77500000
2021-12-27,546.50000000,572.00000000,543.10000000,563.70000000,803238.13100000,803238.13100000
...,...,...,...,...,...,...
2022-06-26,239.70000000,245.20000000,233.20000000,233.90000000,456762.54700000,456762.54700000
2022-06-27,234.00000000,241.00000000,230.00000000,233.80000000,483616.79800000,483616.79800000
2022-06-28,233.90000000,242.80000000,225.90000000,226.90000000,632626.77900000,632626.77900000
2022-06-29,227.00000000,229.80000000,215.40000000,219.70000000,711780.69100000,711780.69100000


In [10]:
daily_close_coins['BNB'] = bnb_df['close']  #get only closing data
daily_close_coins['BNB']

2021-12-23    549.00000000
2021-12-24    542.10000000
2021-12-25    547.40000000
2021-12-26    546.40000000
2021-12-27    563.70000000
                  ...     
2022-06-26    233.90000000
2022-06-27    233.80000000
2022-06-28    226.90000000
2022-06-29    219.70000000
2022-06-30    218.50000000
Name: close, Length: 190, dtype: object

In [11]:
_output_bnb_file = os.path.join(_data_folder, f"BNB_{quote_currency}_daily_candles.csv")
bnb_df.to_csv(_output_bnb_file, index_label='date', mode='w')

In [12]:
df_coinbase = pd.DataFrame(daily_close_coins)
df_coinbase

,BTC,ETH,BNB,SOL,ADA,WLUNA,AVAX,DOT,DOGE,SHIB,MATIC,CRO,LTC,UNI,LINK,ALGO,BCH,XLM,ATOM,AXS
2021-12-23,50842.20,4113.08,549.00000000,190.07,1.4743,94.42,121.37,29.170,0.18510,0.000039,2.6858,0.5951,163.74,18.180,22.190,1.4697,455.93,0.287776,28.380,107.47
2021-12-24,50851.38,4049.65,542.10000000,190.65,1.3935,95.77,114.93,28.170,0.18620,0.000036,2.4687,0.6280,161.35,17.500,21.470,1.5760,452.27,0.278628,26.870,104.62
2021-12-25,50428.31,4097.43,547.40000000,193.06,1.4542,98.39,114.86,28.860,0.19110,0.000038,2.6409,0.6434,158.00,17.490,22.120,1.5748,455.36,0.290047,29.790,109.27
2021-12-26,50801.79,4065.91,546.40000000,198.00,1.4549,99.63,115.07,31.340,0.18990,0.000038,2.8793,0.6233,155.96,18.670,22.990,1.6079,451.86,0.291987,32.170,107.62
2021-12-27,50717.77,4038.64,563.70000000,195.74,1.5164,90.88,113.96,30.970,0.18790,0.000039,2.7267,0.6234,155.78,19.110,23.020,1.6631,465.45,0.300288,29.680,107.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-26,21027.92,1197.13,233.90000000,39.39,0.4899,NaN,19.71,7.832,0.07295,0.000011,0.5584,0.1241,56.58,5.592,6.700,0.3316,112.89,0.120282,8.054,16.17
2022-06-27,20725.17,1191.27,233.80000000,38.50,0.4853,NaN,19.33,7.734,0.07206,0.000011,0.5321,0.1243,55.86,5.330,6.547,0.3310,111.44,0.118877,7.770,15.90
2022-06-28,20252.89,1142.65,226.90000000,35.30,0.4690,NaN,18.08,7.303,0.06602,0.000010,0.5099,0.1199,52.65,4.917,6.303,0.3124,103.03,0.114237,7.340,15.27
2022-06-29,20094.79,1099.15,219.70000000,33.95,0.4658,NaN,17.76,7.029,0.06939,0.000010,0.4968,0.1173,53.63,5.110,6.224,0.3014,104.45,0.110086,7.388,14.38


### Save Single File With Only Closing Data For All Coins Combined

In [13]:
close_candle_output_file = os.path.join(_data_folder, "2022_portfolio_daily_close.csv")
df_coinbase.to_csv(close_candle_output_file,
                   index=True,
                   index_label='date',
                   mode='w',  #python file mode. 'a' is for append, 'w' for write, etc
                   #header=False, #use this when appending, so column names are not written in the middle of the file
                  )

---
---

## Retrieving Historic Data With API From ALPHA VANTAGE
The AlphaVantage section below is going to be turned into markdown so I don't accidentally run it. If you want to use it, simply turn those cells into code cells.

start_date = "2022-01-01"
end_date   = "2022-02-28"

daily_close_candles = {}
problematic_coins = []  #used to collect coins that could not be found in AlphaVantage API

for coin in df['Coin']:
    print(f"--> Parsing data for {coin}")
    
    if coin in daily_close_candles:  #skip coins that are already in the candle dict
        continue
        
    if coin=='WLUNA':
        coin = 'LUNA'  #alphavantage does not support WLUNA, so we used the real token
        print("\t* Changed WLUNA to LUNA")
    candles = get_daily_data(symbol         = coin,
                             api_key        = alpha_vantage_key,
                             start_date     = start_date,
                             end_date       = end_date,
                             quote_currency = 'USD',
                             simplify       = True)
    
    if candles is None:
        problematic_coins.append(coin)
        continue
        
    close_candles = candles['close']  # I only want the closing data
    daily_close_candles[coin] = close_candles
    
    time.sleep(13)  #AlphaVantage allows 5 requests per minute, so I'm delaying between requests

portfolio_df = pd.DataFrame(daily_close_candles)
portfolio_df

output_file = os.path.join(_data_folder, "portfolio_daily_close.csv")
portfolio_df.to_csv(output_file)